In [1]:
import os
import cv2
import glob
import torch
import shutil
import time
import copy
import itertools
import torch.nn as nn
import torch.optim as optim
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from pathlib import Path
from torch.nn import functional as F
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from PIL import Image

In [2]:
# #FOR NORMAL IMAGES
# positive_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_sampled_positive\\'
# negative_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_sampled_negative\\'

In [3]:
#FOR NORMAL W/AUG IMAGES
positive_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\1200aug_equalized_sampled_positive\\'
negative_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\1200aug_equalized_sampled_negative\\'

In [4]:
# #FOR MASKED IMAGES
# positive_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\masked_images\\positive\\'
# negative_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\masked_images\\negative\\'

In [5]:
os.path.exists(negative_path)

True

In [6]:
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model\\working\\train')
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/test')

# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/positive')
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/test/positive')

# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/negative')
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/test/negative')

In [7]:
positive_train_len = int(np.floor(len(os.listdir(positive_path))*0.8))
positive_len = len(os.listdir(positive_path))

negative_train_len = int(np.floor(len(os.listdir(negative_path))*0.8))
negative_len = len(os.listdir(negative_path))

In [8]:
for file in os.scandir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/positive'):
    os.remove(file.path)
for trainimg in itertools.islice(glob.iglob(os.path.join(positive_path, '*.png')), positive_train_len):
    shutil.copy(trainimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/positive')
    
for file in os.scandir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/negative'):
    os.remove(file.path)
for trainimg in itertools.islice(glob.iglob(os.path.join(negative_path, '*.png')), negative_train_len):
    shutil.copy(trainimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/negative')

for file in os.scandir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/val/positive'):
    os.remove(file.path)
for testimg in itertools.islice(glob.iglob(os.path.join(positive_path, '*.png')), positive_train_len, positive_len):
    shutil.copy(testimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/val/positive')

for file in os.scandir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/val/negative'):
    os.remove(file.path)
for testimg in itertools.islice(glob.iglob(os.path.join(negative_path, '*.png')), negative_train_len, negative_len):
    shutil.copy(testimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/val/negative')

In [9]:
normalizer = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        # transforms.ColorJitter(),
        transforms.ToTensor(),
        # transforms.functional.equalize(),
        normalizer
    ]),
    
    'validation': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        normalizer
    ]),

    'test': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        normalizer
    ])
}

In [10]:
data_images = {
    'train': datasets.ImageFolder('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data/equalized_data_model/working/train', data_transforms['train']),
    'validation': datasets.ImageFolder('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data/equalized_data_model/working/val', data_transforms['validation']),
    'test': datasets.ImageFolder('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data/equalized_data_model/working/test', data_transforms['test'])
}

In [11]:
os.path.exists('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data/equalized_data_model/working/test')

True

In [12]:
dataloaders = {
    'train': torch.utils.data.DataLoader(data_images['train'], batch_size=32, shuffle=True, num_workers=0),
    'validation': torch.utils.data.DataLoader(data_images['validation'], batch_size=32,shuffle=True,num_workers=0),
    'test': torch.utils.data.DataLoader(data_images['test'], batch_size=32,shuffle=True,num_workers=0)
}

In [13]:
device_id = 0
torch.cuda.device_count()
torch.cuda.get_device_name(device_id)
# print(device_id)

'NVIDIA GeForce GTX 1660 Ti'

In [14]:
torch.cuda.empty_cache

<function torch.cuda.memory.empty_cache() -> None>

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.hub._validate_not_a_forked_repo=lambda a,b,c: True
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
if torch.cuda.is_available():
    model.cuda()

Using cache found in C:\Users\arnav/.cache\torch\hub\pytorch_vision_v0.10.0


In [16]:
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [17]:
# for param in model.parameters():
#     param.requires_grad = False

In [18]:
model.fc = nn.Sequential(
    nn.Linear(2048, 64),
    nn.ReLU(inplace=True),
    nn.Linear(64, 3)
).to(device)

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

In [20]:
# def trained_model(model, criterion, optimizer, epochs):
#     for epoch in range(epochs):
        
#         print('Epoch:', str(epoch+1) + '/' + str(epochs))
#         print('-'*10)
        
#         for phase in ['train', 'validation']:
#             if phase == 'train':
#                 model.train() #this trains the model
#             else:
#                 model.eval() #this evaluates the model

#             running_loss, running_corrects = 0.0, 0 

#             for inputs, labels in dataloaders[phase]:
#                 inputs = inputs.to(device) #convert inputs to cpu or cuda
#                 labels = labels.to(device) #convert labels to cpu or cuda

#                 outputs = model(inputs) #outputs is inputs being fed to the model
#                 loss = criterion(outputs, labels) #outputs are fed into the model
#                 # loss.requires_grad = True


#                 if phase == 'train':
#                     optimizer.zero_grad() #sets gradients to zero
#                     outputs = model(inputs) #outputs is inputs being fed to the model
#                     loss.backward() #computes sum of gradients
#                     optimizer.step() #preforms an optimization step

#                 _, preds = torch.max(outputs, 1) #max elements of outputs with output dimension of one
#                 running_loss += loss.item() * inputs.size(0) #loss multiplied by the first dimension of inputs
#                 running_corrects += torch.sum(preds == labels.data) #sum of all the correct predictions

#             epoch_loss = running_loss / len(data_images[phase]) #this is the epoch loss
#             epoch_accuracy = running_corrects.double() / len(data_images[phase]) #this is the epoch accuracy

#             print(phase, ' loss:', epoch_loss, 'epoch_accuracy:', epoch_accuracy)

#     return model

In [21]:
# import EarlyStopping
# import pytorchtools
# from pytorchtools import EarlyStopping

In [22]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_precision = 0.0
    best_recall = 0.0
    best_f1 = 0.0
    flag = 0

    for epoch in range(num_epochs):
        if flag == 1:
            break
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            iteration_count=0
            total_precision=0
            total_recall=0
            total_f1=0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                iteration_count+=1
                precision_it= precision_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
                recall_it= recall_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
                f1_it= f1_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
                total_precision+=precision_it
                total_recall+=recall_it
                total_f1+=f1_it

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(data_images[phase])
            epoch_acc = running_corrects.double() / len(data_images[phase])
            epoch_precision=total_precision/iteration_count
            epoch_recall=total_recall/iteration_count
            epoch_f1=total_f1/iteration_count

            print('{} Loss: {:.3f} Acc: {:.3f} Prec: {:.3f} Recall: {:.3f} F1 {:.3f}'.format(
                phase, epoch_loss, epoch_acc, epoch_precision, epoch_recall, epoch_f1))

            # deep copy the model and implement early stopping to prevent overfitting
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_precision = epoch_precision
                best_recall = epoch_recall
                best_f1 = epoch_f1
                es = 0
                best_model_wts = copy.deepcopy(model.state_dict())
            elif phase == 'test':
                es += 1
                print("EarlyStopping counter {} of 3".format(es))

                if es > 300:
                    print('Early stopping with best_acc: {:4f} and test_acc for this epoch: {:.4f}'.format(best_acc,epoch_acc))
                    flag = 1
                

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:.3f}, Best Precision: {:.3f}, Best Recall: {:.3f}, Best F1: {:.3f}'.format(best_acc, best_precision, best_recall, best_f1))


    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [23]:
optimizer_ft = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [24]:
# model = trained_model(model, criterion, optimizer, 1)

In [25]:
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=30)

Epoch 1/30
----------


C:\Users\arnav\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 0.909 Acc: 0.633 Prec: 0.631 Recall: 0.633 F1 0.601
test Loss: 0.520 Acc: 0.755 Prec: 0.789 Recall: 0.750 F1 0.725

Epoch 2/30
----------
train Loss: 0.527 Acc: 0.742 Prec: 0.783 Recall: 0.742 F1 0.730
test Loss: 0.665 Acc: 0.541 Prec: 0.792 Recall: 0.538 F1 0.486
EarlyStopping counter 1 of 3

Epoch 3/30
----------
train Loss: 0.479 Acc: 0.782 Prec: 0.811 Recall: 0.782 F1 0.776
test Loss: 0.612 Acc: 0.790 Prec: 0.815 Recall: 0.792 F1 0.793

Epoch 4/30
----------
train Loss: 0.329 Acc: 0.866 Prec: 0.877 Recall: 0.866 F1 0.865
test Loss: 0.714 Acc: 0.574 Prec: 0.771 Recall: 0.570 F1 0.523
EarlyStopping counter 1 of 3

Epoch 5/30
----------
train Loss: 0.329 Acc: 0.867 Prec: 0.886 Recall: 0.867 F1 0.864
test Loss: 0.439 Acc: 0.807 Prec: 0.831 Recall: 0.809 F1 0.811

Epoch 6/30
----------
train Loss: 0.275 Acc: 0.886 Prec: 0.900 Recall: 0.886 F1 0.885
test Loss: 0.649 Acc: 0.654 Prec: 0.788 Recall: 0.660 F1 0.652
EarlyStopping counter 1 of 3

Epoch 7/30
----------
train Loss: 0

In [26]:
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\alexnet_model_weights/working/models')

In [27]:
torch.save(model.state_dict(), 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\alexnet_model_weights/working/models/weights.h5') #save the model's weights
model.load_state_dict(torch.load('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\alexnet_model_weights/working/models/weights.h5')) #load the model's weights

<All keys matched successfully>